# Explaining Facial Expression Recognition
## Notebook 2:  XAI for Affective Computing with Counterfactuals and SHAP (SoSe2023)

In the first task in this notebook, you will attempt to generate Counterfactual Explanations Facial Expression Recognition using Facial Action Units.  In the second task, you will generate SHAP based feature attribution explations for an image based CNN using the SHAP implementation of the DeepLift algorithm.  For both parts, we will be using the same models and data as the previous notebook.  

To use this notebook, please make sure to go step by step through each of the cells review the code and comments along the way.

Make sure to read the Notebook 2 section of the **README** beforing starting this notebook for all installation instructions.

## Notebook Setup

In [ ]:
%load_ext autoreload
%autoreload 2

##### Import necessary libraries

(see README for necessary package installations if you receive a `module not found` error.

In [ ]:
import pickle
from pathlib import Path

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

# import tensorflow for model loading
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# import sklearn for processing data and results
from sklearn.preprocessing import LabelBinarizer

from skimage import io

# import model loading function
from model import cnn_model, create_bn_replacment

import utils

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')
tf.config.list_physical_devices('GPU')

##### Some Global Variables

In [ ]:
SEED = 12
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 80 # set to 80 to easily load all images using image generator in one call
NUM_CLASSES = 8
CLASS_LABELS = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']

## Task 1: Counterfactuals with Facial Action Units

### Load the Model and Data

#### Load the Random Decision Forest (RDF) Model

Here we will load the pretrained random decision forest (RDF) trained on the facial action units (FAUs) that were extracted from the AffectNet dataset using [OpenFace](https://github.com/TadasBaltrusaitis/OpenFace).

(This is the same model as our previous notebook)

In [ ]:
with open('../models/affect_rdf.pkl', 'rb') as f:
    fer_rdf_model = pickle.load(f)

#### Load the data

Next we will load the preextracted FAUs from a `csv` file created by OpenFace during FAU extraction from the AffectNet dataset.  We load the data into Pandas Dataframes, then convert the columns to numpy array for easier processing with scikit-learn.

The numpy array `X_aus` contains FAUs from the 80 images available for explainations.  And `Y_aus_true` stores the ground truth labels, encoded as [one hot vectors](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) for each set of FAUs.

In [ ]:
# Training data
train_csv = '../data/affectnet_aus/train_aus.csv'
df_aus_train = pd.read_csv(train_csv)

# Small dataset for explanations
xai_csv = '../data/affectnet_aus/eval_aus.csv'
df_aus_xai = pd.read_csv(xai_csv)

# get only the columns storing action units from the dataframe
feature_cols = [col for col in df_aus_xai if col.startswith('AU')]

X_aus = np.array(df_aus_xai.loc[:, feature_cols])
Y_aus_true = np.array(df_aus_xai['class'])

print('XAI Dataset', X_aus.shape, Y_aus_true.shape)

#### Evaluate the model

Now let's evaluate the performance of the RDF Classifier on on the `X_aus` dataset. The accuracy should be around $42\%$

- This is the same dataset for the last notebook. If you want to review more results (such as full test data or confusion matrices), please review your previous notebook.

We should also generate the predictions of the model for the dataset, and store them in `Y_aus_pred`.

In [ ]:
# get model predictions
print(f'{fer_rdf_model.score(X_aus, Y_aus_true) * 100:0.2f}% Accuracy')

#### Generate model predictions for dataset

In [ ]:
Y_aus_pred = fer_rdf_model.predict(X_aus)

### Review the Dataset

In [ ]:
# displays first 9 images in array
start = 40

# Gets all images from folder
fau_images = [io.imread(f) for f in df_aus_xai.image]

# gets labels for ground truth and predictions
true_labels = [CLASS_LABELS[idx] for idx in Y_aus_true]
pred_labels = [CLASS_LABELS[idx] for idx in Y_aus_pred]

utils.display_nine_images(fau_images, true_labels, pred_labels, start)

### Identify Some Images to Explain
- Review the FAU dataset using the helper code in above.
- Change the start value to get a new set of images (there are 10 images for each class, so for example, the class happy will be at indexes 10-19)
- Search through the images to find at least 4 to explain 
    - Find classes that you would like to explain, and from each class select 2 images
        - one should be a correct prediction  
        - and one should be an incorrect prediction
    - For each image, also choose the desired class index that you would like to generate counterfactuals for
        - make sure to think about what is important for a desired class based on the prediction results

In [ ]:
#### Enter the Indexes Here ### 
###############################
# you will use these arrays later in the task
img_idxs = []
desired_classes = []

## Task 1: Generate Explanations with DiCE
In this part of the notebook, you will generate Counterfactual Explanations using the Python Library, [Diverse Counterfactual Explanations (DiCE)](http://interpret.ml/DiCE/). Make sure to read the documentation and getting started information.

Counterfactual explanations typically work best on tabular data, so in this part we will are using the FAU dataset with the RDF model.

In [ ]:
# DiCE imports
import dice_ml
from dice_ml.utils import helpers  # helper functions

pd.set_option('display.max_columns', None) # so that Jupyter doesn't truncate columns of dataframe

In [ ]:
# identify categorical and numerical features as needed by DiCE
categorical_features = list(df_aus_train.columns[df_aus_train.columns.str.contains('_c')])
numerical_features = list(df_aus_train.columns[df_aus_train.columns.str.contains('_r')])

# convert categorical features to strings as required by DICE
df_aus_train[categorical_features] = df_aus_train[categorical_features].astype('str')
df_aus_xai[categorical_features] = df_aus_xai[categorical_features].astype('str')
df_aus_train[categorical_features] = df_aus_train[categorical_features].astype('category')
df_aus_xai[categorical_features] = df_aus_xai[categorical_features].astype('category')

all_features = numerical_features + categorical_features 

### Task 1.1 
In this task, you will use DiCE to generate a set counterfactual explanations for your selected instances.

#### Task 1.1.1 Setup a DiCE explainer instance

See the [intro to DiCE](http://interpret.ml/DiCE/notebooks/DiCE_getting_started.html) for details on working with this library.

Note: DiCE requires requires pandas dataframes for creating explainers and explanations. 
- for setting up the explainer you can use the following to create a dataframe of features and classes from the training data
    - `df_aus_train[all_features+['class']`
- for generating instances to explain, you can use the following code:
    - `df_aus_xai[all_features][40:41]` where 40 is the index of instance to explan

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 1.1.2: Use the Explainer to Generate Counterfactual Explanations

Generate counterfacutal explanations for each of your select data instances from task 3.0.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 1.1.3: Visualize Counterfactuals

Now visualize the counterfactuals as Pandas dataframes. 

It would also be helpful to include the original image with the explanation, as well as to print the label names for the ground truth, the prediction, and the desired outcome.  

In [ ]:
##### YOUR CODE GOES HERE #####
###############################

#### Task 1.1.4: Describe your observations

![Action Units](assets/fac.jpg)

1. Which features are most important for the detection of the specific facials expressions of your data instances?  Do the counterfactuals make sense according to your intuition of the contrastive expression class you're using?
2. The `generate_counterfactuals` method has a parameter `features_to_vary` so that we can restrict which features are perturbed in CF generation.  Are there any AUs that shouldn't be perturbed for our task of emotion detection? Why or why not? Additionally, should we set `permitted_range` parameter to limit the ranges of our continous features?

### Task 1.2 Generate Feature Attribution Scores from Counterfactuals

DiCE can also generate [local and global feature attribution scores](http://interpret.ml/DiCE/notebooks/DiCE_getting_started.html#Generating-feature-attributions-(local-and-global)-using-DiCE) based on the identified counterfactuals.  In this task, we will do just that.  

In [ ]:
# function for plotting importance dictionaries provided by DiCE
def plot_importance_dict(importance_dict):
    keys = list(importance_dict.keys())
    vals = [float(importance_dict[k]) for k in keys]
    sns.barplot(x=keys, y=vals)
    plt.xticks(rotation=45)

#### Task 1.2.1 Generate and Plot Local Importance Scores

Using your previously defined DiCE explainer, generate and plot (with the help of the function above) local importance scores your your data instances.

Again, it is helpful to also include the original image and FAU values, as well as to print the label names for the ground truth, the prediction, and the desired outcome.   

In [ ]:
##### YOUR CODE GOES HERE #####
###############################
from IPython.display import display # (use display to display a dataframe)



#### Task 1.2.2 Generate and Plot Local Importance Scores

Using your previously defined DiCE explainer, generate and plot global importance using the entire XAI dataset.

In [ ]:
##### YOUR CODE GOES HERE #####
###############################


#### Task 1.2.3 Describe your findings

![Action Units](./assets/fac.jpg)

1. Based on the DiCE documentation, how does DiCE calculate feature importance from counterfactuals?
2. Do the plots lead to any interesting insights regarding AUs or facial expression detection?

Write your answer here...

## Task 2 - SHAP Explanations and Measuring Interpretability

In this part of the Notebook, you will generate Shaply Value based saliency map explanations for predictions from the FER CNN from the last notebook. To do this, you will using the [SHAP Python Package](https://shap.readthedocs.io/en/latest/index.html), which is an extensive implementation of explantion methods using shapley values.  For this section, we will focus on just the CNN based explanation method.  

We will also perform a "sanity check" of the SHAP output using the methodolgy we learned about in the ["Sanity Checks for Saliency Maps" paper](https://lernraumplus.uni-bielefeld.de/mod/folder/view.php?id=810572).  

Before getting started with this task make sure to review the documention of SHAP.

### Load the Model and Data
Here we are loading the pretrained Convolutional Neural Network for Facial Expression Recognition (FER) trained on raw images from a subset of the [AffectNet dataset](http://mohammadmahoor.com/affectnet/). 

This is the same model as our previous notebook 

#### Load the CNN model 


In [ ]:
# make sure you've downloaded the models from LernraumPlus (see README instructions for Notebook I)
model_path = '../models/affectnet_model_e=60/affectnet_model'

# test loading weights
fer_cnn_model = cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
fer_cnn_model.load_weights(model_path)

#### Load the data
`ImageDataGenerator` is a [Keras utility class](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) to easily load images for processing with a Keras model.

The numpy array `X_img` contains 80 images that we will use for explanations.  And `Y_img_true` stores the ground truth labels, encoded as [one hot vectors](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/), for each image.

In [ ]:
test_dir = '../data/affectnet/val_class/'
# test_dir = '../localdata/affectnet/val_class/'


# Load data
test_datagen = ImageDataGenerator(validation_split=0.2,
                                  rescale=1./255)
test_gen = test_datagen.flow_from_directory(directory=test_dir,
                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                            batch_size=BATCH_SIZE,
                                            shuffle=False,
                                            color_mode='rgb',
                                            class_mode='categorical', 
                                            seed = SEED)
X_img, Y_img_true = next(test_gen)

#### Evaluate model
Next we will evaluate the loaded model to ensure it is working as expected.  You should get around $48.75\%$ accuracy. While this is not a perfect classifier is well above random guessing which is $1 / 8 * 100 = 12.5$ accuracy

This is the same CNN model as before, so refer to our previous notebook to view more details on its performance.

We also generate the predictions of the model for the dataset, and store them in `Y_img_pred`.

In [ ]:
loss, acc = fer_cnn_model.evaluate(test_gen, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

#### Generate model predictions for dataset

In [ ]:
Y_img_pred = fer_cnn_model.predict(X_img)

### Review the Dataset

In [ ]:
# displays first 9 images in array
start = 0

# gets labels for ground truth and predictions
true_labels = [CLASS_LABELS[idx] for idx in np.argmax(Y_img_true, axis=1)]
pred_labels = [CLASS_LABELS[idx] for idx in np.argmax(Y_img_pred, axis=1)]

utils.display_nine_images(X_img, true_labels, pred_labels, start)

### Identify Some Images to Explain
- Review the FAU dataset using the helper code above.
- Try changing start value to get a new set of images (there are 10 images for each class, so for example, the class happy will be at indexes 10-19)
- Search through the images to find at least 4 to explain 
    - Find classes that you would like to explain, and from each class select 2 images
        - one should be a correct prediction  
        - and one should be an incorrect prediction

In [ ]:
#### Enter the Indexes Here ### 
###############################
# you will use this array later in the task
img_idxs = []

### Task 2.1: DeepLift based Explanations with SHAP

In this task we will generate feature attribution explanations for our AffectNet CNN using SHAP and its implementation of an enhanced version of DeepLIFT, called [DeepExplainer](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html). To better understand the method you can review the [API documentation](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.DeepExplainer.html).

Unfortunatly, one of the downsides to this approach is computation time. If you're not running this notebook on a GPU you may have to be a bit patient when calculating the SHAP  values. 

In [ ]:
# First we have to import shap and add a few fixes for our model
import shap

# fixes issues with running deep explainer on our model
# https://github.com/slundberg/shap/issues/1761
shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.linearity_1d(0)

#### Task 2.1.1 Generate SHAP Values 

Generate the SHAP values for each of the four images you selected above.  You can review the [DeepExplainer Tutorial](https://shap-lrjball.readthedocs.io/en/latest/example_notebooks/deep_explainer/Front%20Page%20DeepExplainer%20MNIST%20Example.html) for help

In [ ]:
# the shap deep explainer accepts a batch of input to generate 
# shap values for all inputs in one call the generate method 
images_xai = X_img[img_idxs]
labels_true_xai = np.array(true_labels)[img_idxs]
labels_pred_xai = np.array(pred_labels)[img_idxs]

#### Your Code Here ####
########################



#### Tasks 2.1.2 Visualize the Explanations

Now that we have calculated the SHAP values, lets visualize them. Use the SHAP API, from the above references, to plot the shap values as images.  Instead of printing the only the true label with the original image, print the true and predicted labels.

#### Task 2.1.3 Evaluate SHAP Values
One of the main features of SHAP values is that they sum up to the difference between the expected model output (i.e. the average of the predictions on the 'background sample') and the current output.

In other words,

$$
   \Sigma shap(x) = f(x) - E[f(X)]
$$

where $f(x)$ is our model, $x$ is the current instance being explained, and $X$ is the background sample (in our case the data in `X_xai`)

Using the generated SHAP values and the generated model predictions, show that this is true for the Deep Explainer.

In [ ]:
#### Your Code Here ####
########################



### Task 2.2 Evaluate SHAPs Invariance to Model Changes

In the paper "Sanity Checks for Saliency Maps" we saw the authors validated feature attribution methods by testing their invariance to models with randomly initialized weights (i.e. models that have not been trained). We will do exactly that in this task to evaluate SHAP's DeepExplainer method. 

#### Task 2.2.1 Create Randomly Intialized Model

First let's see what SHAP explanations for a model with completely randomized weights look like. 

This can be done by loading the CNN model without loading the saved weights (see the section on loading the data for code examples).  Once the model is loaded generate the predictions for our X_img dataset, as this will be needed to generate the SHAP explanations.

In [ ]:
#### Your Code Here ####
########################

# replace None with the code to load the model
fer_model_cnn_rnd = None

#### Task 2.2.2 Generate SHAP Values for Randomized Model

Now generate the SHAP values for the random model and predictions, then visualize them as we did in tasks 2.1.1 and 2.1.2.  

In [ ]:
#### Your Code Here ####
########################



#### Task 2.2.3 Cascading Randomization

Additionally in the paper, the authors evaluated the methods' invariance to randomized layers in a cascading fashion. In this task, we will also do that for SHAP.

Using the code below as a template, generate SHAP values for a **single image**, from your selected images, as the weights of each layer are randomized.  Once you've generated SHAP values for each randomized layer, plot them similarly to the paper, including the name of the layer as the title for the column.

In [ ]:
# Check Randomized model for just the first image in your selection (for the sake of time)
img_idx = 0
img = X_img[img_idxs[img_idx]:img_idxs[img_idx]+1]

# load a new model with pretrained weights
fer_cnn_seqrnd = cnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)
fer_cnn_seqrnd.load_weights(model_path)

# dictionary for storing randomized layer name and the shap values
shap_vals_seq = {}

#### Your Code Here ####
########################
# generate shap values for the original model (before randomizing) and save to the dictionary
# only save the shap values for the predicted class

# loop through the model layers in reverse and randomize each layer's weights as we go
for i in range(1, len(fer_cnn_seqrnd.layers)):
    if hasattr(fer_cnn_seqrnd.layers[-i], 'kernel_initializer'):
        fer_cnn_seqrnd.layers[-i].set_weights(fer_model_cnn_rnd.layers[-i].weights) # copy weights from random model into current model
        layer_name = fer_cnn_seqrnd.layers[-i].name
        
        #### Your Code Here ####
        ########################
        # generate shap values for each reinitialized layer and save to dictionary
        # only save the shap values for the predicted class



##### Task 2.2.4

Based on the above results would you consider SHAP invariant to model randomization? Why or why not? Furthermore, would you consider SHAP to be better than an Edge Detector?

### Bonus Task - Evaluate GradCAM for model invariance

For a bonus task, perform the same evaluations of model invariance using our implementation of GradCAM from the last notebook (or any other feature map method).